In [ ]:
#import required modules
import networkx as nx
import numpy as np 
import math 
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time 
from random import sample

In [ ]:
def findAllLatentNeighbours(Graph,nodeNum, withoutNeighbours = False):
    
    #find all neighbours
    neighbours = list(Graph.neighbors(nodeNum))
    
    #find all latent neighbours 
    all_neighbours = []
    
    #for each of the neighbour nodeNumbers returned,find the neighbour and append it to list 
    for i in neighbours:
        all_neighbours.append(Graph.neighbors(i))
        
    
    latent_neighbours = set()
    
    for i in all_neighbours:
        for j in i:
            latent_neighbours.add(j)
            
    if(nodeNum in latent_neighbours):
        latent_neighbours.remove(nodeNum)
        
    if(withoutNeighbours):
        latent_neighbours = [i for i in latent_neighbours if i not in neighbours]
            
    latent_neighbours = list(latent_neighbours)
        #return list 
    return latent_neighbours

In [ ]:
#common enighbours
def findCommonNeighbours(G,node1,node2):
    if node1 not in G:
        return []
    if node2 not in G:
        return []
    
    return sorted(list(nx.common_neighbors(G,node1,node2)))


In [ ]:
#calculate neighbourhood vector 
def neighbourhoodVector(Graph,nodeNum,func = findCommonNeighbours):
    nodes = Graph.nodes
    
    neighbourhoodVec = np.zeros(Graph.number_of_nodes() + 1).astype(float)
    #makes a size len(node)
    #will slice the first nodes off
    
    #check if it is an isolated node
    try:
        neighbours = list(nx.neighbors(Graph,nodeNum))
    except nx.NetworkXError:
        neighbours = []

    #find all the latent neighbours that are not direct neighbours
    latent_neighbour = findAllLatentNeighbours(Graph,nodeNum, True)
    
    for i in range(1, Graph.number_of_nodes() + 1):
        if (i == nodeNum):
            neighbourhoodVec[i] = Graph.degree[i]
        elif (i in neighbours):
            if func == findCommonNeighbours:
                neighbourhoodVec[i] = len(func(Graph,nodeNum,i)) + 1
            else:
                neighbourhoodVec[i] = func(Graph,nodeNum,i) + 1
        elif (i in latent_neighbour):
            if func == findCommonNeighbours:
                neighbourhoodVec[i] = len(func(Graph,nodeNum,i)) 
            else:
                neighbourhoodVec[i] = func(Graph,nodeNum,i)

        else:
            neighbourhoodVec[i] = 0
            
    return neighbourhoodVec[1:]


In [ ]:
def unionNeighbourhood(neighbourhoodVec1,neighbourhoodVec2):
    unionVector = set()
    
    for i in range(0,len(neighbourhoodVec1)):
        if neighbourhoodVec1[i] > 0:
            unionVector.add(i+1)
            
    for i in range(0,len(neighbourhoodVec2)):
        if neighbourhoodVec2[i ] > 0:
            unionVector.add(i+1)
                        
    if (len(unionVector) == 0):
        return []
    else:
        temp = list(unionVector)
        temp.sort()
        return temp


In [ ]:
def NAvg(neighbourhoodVec, unionVec):
    
    if (unionVec.size == 0):       #we send in an array 
        return 0

    count = 0
    for i in unionVec:
        count = count + neighbourhoodVec[i-1]
        
    return count / (len(unionVec))

In [ ]:
def correlation(G,node1,node2,func = findCommonNeighbours):
    
    N_i = neighbourhoodVector(G,node1,func)
    N_j = neighbourhoodVector(G,node2,func)
    UN_ij = np.asarray(unionNeighbourhood(N_i,N_j))
    
    N_avg_i= NAvg(N_i, UN_ij)
    N_avg_j= NAvg(N_j, UN_ij)
        

    summation1 = 0
    summation2 = 0
    summation3 = 0
    
    for i in UN_ij:
        summation1 = summation1 + (N_i[i-1] - N_avg_i) * (N_j[i-1] - N_avg_j)
        summation2 = summation2 + (N_i[i-1] - N_avg_i) **2 
        summation3 = summation3 + (N_j[i-1] - N_avg_j) **2 
        
        
    b = (math.sqrt(summation2) * math.sqrt(summation3))
    
    if(b != 0):
        a = summation1 / (b)
    else:
        a = 0
    
    return a


In [ ]:
def locallyAdaptiveSimilarityMessure(G,nodeNum1,nodeNum2):
    
    #find common neighbours 
    commonNeighbours = findCommonNeighbours(G,nodeNum1,nodeNum2)
    
    if (len(commonNeighbours) == 0 ):
        return 0
    
    #define a_g
    
    all_nodes = list(G.nodes)

    summation_g = 0
    summation_l = 0

    for i in all_nodes:
      summation_g = summation_g + G.degree[i]

    #local degree
    for i in commonNeighbours:
        summation_l = summation_l + G.degree[i]

    a_local = math.log(summation_l/ len(commonNeighbours))
    a_global = math.log(summation_g/ len(all_nodes))

    
    summation = 0
    #for each of the common neighbours 
    for i in commonNeighbours:
        degree = G.degree[i]
        summation = summation + 1/(degree**((a_global/a_local)))

    return summation
    

In [ ]:
def DICN(G,node1,node2, func = findCommonNeighbours):
    return (1 + locallyAdaptiveSimilarityMessure(G,node1,node2)) * (1 + correlation(G,node1,node2,func))

In [ ]:
def computeAUC(G, b):
    
    all_edges = list(G.edges)    
    amount = 1 - (b/100)
    
    training_set, testing_set = train_test_split(all_edges,test_size= amount)       
    
    training_G = nx.Graph()
    training_G.add_edges_from(training_set)
    
    testing_G = nx.Graph()
    testing_G.add_edges_from(testing_set)
    
    nonexisting_edges = sample(list(nx.non_edges(G)),min(testing_G.number_of_edges(),100))

    
    #adding nodes 
    
    for i in list(G.nodes):
        training_G.add_node(i)
        testing_G.add_node(i)

    n1 = 0
    n2 = 0 

    #pick and edge from testing set 
    count = 0
    for i in testing_set:
        #compute similairty 
        similairty_score1 = DICN(training_G,i[0],i[1], func = locallyAdaptiveSimilarityMessure)
        #pick and edge from the non existant set
        for j in nonexisting_edges:
            #computer similairty
            similairty_score2 = DICN(training_G, j[0],j[1], func = locallyAdaptiveSimilarityMessure)
            count = count + 1

            if similairty_score1 > similairty_score2:
                n1 = n1 + 1
            if similairty_score1 == similairty_score2:
                n2 = n2 + 1
    
    #n = (1 − β/100) · |E| · (|V|·(|V|−1)/ 2 − |E|)
#     n = (1 - b/100) * len(edges)*((34*(34-1))/2 - len(edges))
#     AUC = (n1 + n2/2)/(n)

    b = len(testing_set) * len(nonexisting_edges)
    if ( b != 0):
        AUC = (n1 + n2/2)/(b)
    else: 
        AUC = 0

    return AUC

In [ ]:
def avgAUC(G, b,n):
    auc_avg = 0 
    c = time.time()
    for i in range(0,n):
        a = computeAUC(G,b)
        auc_avg = auc_avg + a
        print("AUC" + str(i) + ": " + str(a))
    print("Time: ", time.time() - c )

            
    return auc_avg/n

In [ ]:
#open file

def openFile(name, ext, ignore_char, readLen):

    edges = []
    with open(name+"."+ext) as reader:
        lines = reader.readlines()
        for i in lines:
            if i[0] != ignore_char:
                a = i.split()
                if (len(a) == readLen):
                    edges.append((int(a[0]),int(a[1])))
                    
    return edges
                

In [ ]:
def nodes_with_no_common_neighbours(G,edges):
  max_edges = max(list(G.nodes))
  no_common_neighbours = []

  no_common_edges = []

  for i in range(1,max_edges + 1):
      for j in range(i+1,max_edges + 1):
        a = list(nx.common_neighbors(G,i,j))
        if not a:
          no_common_neighbours.append((i,j))

  for i in edges:
    if G.degree[i[0]] > 1 and G.degree[i[1]] > 1:
      if i in no_common_neighbours:
        no_common_edges.append(i)

  return no_common_edges


In [ ]:
def compute_AUC_for_no_common_Neighbours(G, b):
    
    all_edges = list(G.edges)    
    amount = 1 - (b/100)
    
    training_set, testing_set = train_test_split(all_edges,test_size= amount)       
    
    training_G = nx.Graph()
    training_G.add_edges_from(training_set)
    
    testing_G = nx.Graph()
    testing_G.add_edges_from(testing_set)
    
    nonexisting_edges = sample(list(nx.non_edges(G)),testing_G.number_of_edges)
    #adding nodes 
    
    for i in list(G.nodes):
        training_G.add_node(i)
        testing_G.add_node(i)

    n1 = 0
    n2 = 0 

    #edges that have no common neighbours

    no_common_testing_set = nodes_with_no_common_neighbours(training_G,testing_set)
    no_common_nonexistant_edges = nodes_with_no_common_neighbours(training_G,nonexisting_edges)

    if(len(no_common_testing_set)) == 0:
      return -1

    #pick and edge from testing set 
    count = 0
    for i in no_common_testing_set:
        #compute similairty 
        similairty_score1 = DICN(training_G,i[0] ,i[1], func = locallyAdaptiveSimilarityMessure)
        #pick and edge from the non existant set
        for j in no_common_nonexistant_edges:
            #computer similairty
            similairty_score2 = DICN(training_G, j[0],j[1], func = locallyAdaptiveSimilarityMessure)

            count = count + 1

            if similairty_score1 > similairty_score2:
                n1 = n1 + 1
            if similairty_score1 == similairty_score2:
                n2 = n2 + 1
    
    #n = (1 − β/100) · |E| · (|V|·(|V|−1)/ 2 − |E|)
#     n = (1 - b/100) * len(edges)*((34*(34-1))/2 - len(edges))
#     AUC = (n1 + n2/2)/(n)


    b = len(no_common_testing_set) * len(no_common_nonexistant_edges)
    if ( b != 0):
        AUC = (n1 + n2/2)/(b)
    else: 
        AUC = 0

    return AUC

In [ ]:
def avgAUC_no_common_neighbour(G, b,n):
    auc_avg = 0 
    c = time.time()
    count = 0 
    
    while (count < n):
      a = compute_AUC_for_no_common_Neighbours(G,b)
      if (a == -1):
        count = count
      else:
        auc_avg = auc_avg + a
        count = count + 1 
        print("AUC" + str(count) + ": " + str(a))
   
    print("Time: ", time.time() - c )

            
    return auc_avg/(n)

True
